In [8]:
import shutil
import os

output_dir = r"C:\Users\prazw\Sarangi\output"
# output_dir = "/DDSP/Sarangi/"
# if os.path.exists(output_dir):
#     shutil.rmtree(output_dir)
# os.makedirs(output_dir)
print(os.listdir(r'C:\Users\prazw\Sarangi')) 
# import os
print(os.getcwd())  # Prints the current working directory of Jupyter Notebook


['.ipynb_checkpoints', 'DDSP.ipynb', 'Sarangi.wav']
C:\Users\prazw\Sarangi


In [9]:
!ddsp_prepare_tfrecord \
  --input_audio_filepatterns=C:\Users\prazw\Sarangi\Sarangi.wav \
  --output_tfrecord_path=C:\Users\prazw\Sarangi\output\Sarangi \
  --num_shards=1 \
  --sample_rate=16000 \
  --frame_rate=50 \
  --example_secs=4 \
  --hop_secs=1 \
  --center=True \
  --viterbi=True

I0106 08:29:25.595705 15120 environments.py:376] Default Python SDK image for environment is apache/beam_python3.10_sdk:2.46.0
I0106 08:29:25.850313 15120 translations.py:710] ==================== <function annotate_downstream_side_inputs at 0x000001FA70434AF0> ====================
I0106 08:29:25.852319 15120 translations.py:710] ==================== <function fix_side_input_pcoll_coders at 0x000001FA70434C10> ====================
I0106 08:29:25.852319 15120 translations.py:710] ==================== <function pack_combiners at 0x000001FA70435120> ====================
I0106 08:29:25.854325 15120 translations.py:710] ==================== <function lift_combiners at 0x000001FA704351B0> ====================
I0106 08:29:25.854325 15120 translations.py:710] ==================== <function expand_sdf at 0x000001FA70435360> ====================
I0106 08:29:25.856331 15120 translations.py:710] ==================== <function expand_gbk at 0x000001FA704353F0> ====================
I0106 08:29:25.85

In [ ]:
tfrecord_file = r'C:\Users\prazw\Sarangi\output\Sarangi-00000-of-00001'
model_dir = r'C:\Users\prazw\Sarangi\model'
train_dir = r'C:\Users\prazw\Sarangi\train'
!ddsp_run \
    --mode=train \
    --alsologtostderr \
    --save_dir={model_dir} \
    --train_tfrecord_path={tfrecord_file} \
    --gin_file=ddsp/training/gin/models/solo_sax.gin \
    --gin_file=ddsp/training/gin/datasets/tfrecord.gin \
    --gin_param="TFRecordProvider.file_pattern='{tfrecord_file}'" \
    --gin_param="batch_size=16" \
    --gin_param="train_util.train.num_steps=1000"

In [ ]:
!ddsp_run \
  --gin_file="C:/Users/prazw/Sarangi/model/solo_instrument.gin" \
  --gin_file="C:/Users/prazw/Sarangi/model/ae.gin" \
  --gin_file="C:/Users/prazw/Sarangi/datasets/tfrecord.gin" \
  --gin_file="C:/Users/prazw/Sarangi/eval/basic_f0_ld.gin" \
  --gin_param="train.total_training_steps=1000" \
  --gin_param="train.batch_size=8" \
  --gin_param="train.checkpoint_steps=500" \
  --gin_param="train.learning_rate=0.0001" \
  --gin_param="train.learning_rate_schedule='linear_decay'" \
  --logdir="C:/Users/prazw/Sarangi/logs" \
  --model_dir="C:/Users/prazw/Sarangi/model" \
  --tfrecord_dir="C:/Users/prazw/Sarangi/output"


In [1]:
# Load TensorBoard extension
%load_ext tensorboard

# Start TensorBoard
%tensorboard --logdir="C:/Users/prazw/Sarangi/logs"

In [ ]:
!ddsp_run \
  --mode=eval \
  --alsologtostderr \
  --save_dir="C:/Users/prazw/Sarangi/evaluation/" \
  --gin_file="C:/Users/prazw/Sarangi/datasets/tfrecord.gin" \
  --gin_file="C:/Users/prazw/Sarangi/eval/basic_f0_ld.gin" \
  --gin_file="C:/Users/prazw/Sarangi/model/operative_config-0.gin" \

In [ ]:
#@title #Install and Import

#@markdown Install ddsp, define some helper functions, and download the model. This transfers a lot of data and _should take a minute or two_.
print('Installing from pip package...')
!pip install -qU ddsp==1.6.5 "hmmlearn<=0.2.7"

# Ignore a bunch of deprecation warnings
import warnings
warnings.filterwarnings("ignore")

import copy
import os
import time

import crepe
import ddsp
import ddsp.training
from ddsp.training import metrics
from ddsp.colab.colab_utils import (
    auto_tune, get_tuning_factor, download, 
    play, record, specplot, upload, 
    DEFAULT_SAMPLE_RATE)
from ddsp.training.postprocessing import (
    detect_notes, fit_quantile_transform
)
import gin
from google.colab import files
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pickle
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

# Helper Functions
sample_rate = DEFAULT_SAMPLE_RATE  # 16000


print('Done!')

In [1]:
import os
import numpy as np
import librosa
import soundfile as sf
import ddsp
import ddsp.training
from ddsp.training import metrics
import gin
import tensorflow.compat.v2 as tf
# from ddsp.training.postprocessing import (
#     detect_notes, fit_quantile_transform
# )

In [3]:
input_audio_path = r"C:\Users\prazw\Sarangi\test\test.wav"
output_audio_path = r"C:\Users\prazw\Sarangi\test\test_output.wav"
model_path = r"C:\Users\prazw\Sarangi\model"
audio, sr = librosa.load(input_audio_path, sr=16000)
print(f"Audio shape: {audio.shape}")
print(f"Sample rate: {sr}")

Audio shape: (250742,)
Sample rate: 16000


In [7]:
#If the audio is mono (len(audio.shape) == 1), reshape it for processing
if len(audio.shape) == 1:
  audio = audio[np.newaxis, :] 

In [8]:
# Setup the session.
ddsp.spectral_ops.reset_crepe()
# Compute features.
audio_features = metrics.compute_audio_features(audio)
# audio_features['loudness_db'] = audio_features['loudness_db'].astype(np.float32)
audio_features['loudness_db'] = audio_features['loudness_db'].numpy().astype(np.float32)

C:\Users\prazw\anaconda3\envs\ddsp\lib\site-packages\librosa\core\convert.py:1870: RuntimeWarning: divide by zero encountered in log10
  + 2 * np.log10(f_sq)


In [10]:
#Load Model
gin_file = os.path.join(model_path, 'operative_config-0.gin')
with gin.unlock_config():
  gin.parse_config_file(gin_file, skip_unknown=True)

# ckpt = os.path.join(model_path, "\ckpt-3300.data-00000-of-00001")

In [13]:
# Ensure dimensions and sampling rates are equal
time_steps_train = gin.query_parameter('F0LoudnessPreprocessor.time_steps')
n_samples_train = gin.query_parameter('Harmonic.n_samples')
hop_size = int(n_samples_train / time_steps_train)
time_steps = int(audio.shape[1] / hop_size)
n_samples = time_steps * hop_size

In [14]:
gin_params = [
    'Harmonic.n_samples = {}'.format(n_samples),
    'FilteredNoise.n_samples = {}'.format(n_samples),
    'F0LoudnessPreprocessor.time_steps = {}'.format(time_steps),
    'oscillator_bank.use_angular_cumsum = True',  # Avoids cumsum accumulation errors.
]

with gin.unlock_config():
  gin.parse_config(gin_params)

In [15]:
# Trim all input vectors to correct lengths 
for key in ['f0_hz', 'f0_confidence', 'loudness_db']:
  audio_features[key] = audio_features[key][:time_steps]
audio_features['audio'] = audio_features['audio'][:, :n_samples]

In [17]:
# Set up the model just to predict audio given new conditioning
model = ddsp.training.models.Autoencoder()
model.restore(model_path)

In [18]:
# Build model by running a batch through it.
_ = model(audio_features, training=False)

C:\Users\prazw\anaconda3\envs\ddsp\lib\site-packages\librosa\core\convert.py:1870: RuntimeWarning: divide by zero encountered in log10
  + 2 * np.log10(f_sq)


In [ ]:
# if ADJUST and DATASET_STATS is not None:
#   # Detect sections that are "on".
#   mask_on, note_on_value = detect_notes(audio_features['loudness_db'],
#                                         audio_features['f0_confidence'],
#                                         threshold)

In [20]:
from IPython.display import Audio
import numpy as np

# Assuming audio_gen is a NumPy array or tensor
audio_gen = model.get_audio_from_outputs(_)
# Convert to NumPy array if it's a TensorFlow tensor
if hasattr(audio_gen, 'numpy'):
    audio_gen = audio_gen.numpy()

# Normalize audio to prevent clipping (optional)
audio_gen = audio_gen / np.max(np.abs(audio_gen))

# Play audio in Jupyter Notebook
Audio(audio_gen, rate=16000)  # Replace 16000 with the correct sample rate

In [19]:
# outputs = model(audio_features, training=False)
audio_gen = model.get_audio_from_outputs(_)
print('Resynthesis')
play(audio_gen)

Resynthesis


NameError: name 'play' is not defined